<h2>Group project</h2>
<p>Performed by</p>
<ol>
    <li>Mikhail Mironov u211361</li>
    <li>Vera Garmanova</li>
</ol>

In [1]:
import requests
import pickle
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver

<h3>Load initial dataset from kaggle</h3>

In [2]:
# load data from kaggle
df_links = pd.read_csv("data/data_with_links.csv") # -> data with linkedin profile links
df_links.head()

,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable,prev_rank,CEO,Website,Ticker,Market Cap,username,profile_links
0,Walmart,1,0.0,572754.0,13673.0,2300000.0,Retailing,Bentonville,AR,no,no,no,yes,1.0,C. Douglas McMillon,https://www.stock.walmart.com,WMT,352037,douglasmcmillon,https://www.linkedin.com/in/dougmcmillon
1,Amazon,2,0.0,469822.0,33364.0,1608000.0,Retailing,Seattle,WA,no,no,no,yes,2.0,Andrew R. Jassy,www.amazon.com,AMZN,1202717,andrewjassy,https://www.linkedin.com/in/andy-jassy-8b1615
2,Apple,3,0.0,365817.0,94680.0,154000.0,Technology,Cupertino,CA,no,no,no,yes,3.0,Timothy D. Cook,www.apple.com,AAPL,2443962,timothycook,https://www.linkedin.com/in/tim-cook-7a188782
3,CVS Health,4,0.0,292111.0,7910.0,258000.0,Health Care,Woonsocket,RI,no,no,yes,yes,4.0,Karen Lynch,https://www.cvshealth.com,CVS,125204,karenlynch,https://www.linkedin.com/in/karenslynch
4,UnitedHealth Group,5,0.0,287597.0,17285.0,350000.0,Health Care,Minnetonka,MN,no,no,no,yes,5.0,Andrew P. Witty,www.unitedhealthgroup.com,UNH,500468,andrewwitty,NaN


In [3]:
# Filter out those who dont have linkedin account, i.e profile_link is NaN
df_links = df_links[df_links.profile_links.notna()]
print(df_links.shape)

# Get user_handle on linkedin as a last element in profile_link string
df_links["handle"] = df_links.profile_links.str.split("/").apply(lambda x: x[-1])

(607, 20)


In [4]:
# expand kaggle data with collected data on experience of ceos
df_exp = pd.read_csv("data/data_ceo_experience.csv")
df_exp.head()

,Handle,CEO years,CEO months,Overall Comp. years,Overall Comp. months,Other CEO years,Other CEO months,Other Comp. years,Other Comp. months,Followers,BCs,MA,PHD,IS_HARVARD,Num Comp.
0,dougmcmillon,13,10,13,10,3,6,3,6,1 136 161,Business,Business,NaN,0.0,2
1,andy-jassy-8b1615,25,8,25,8,0,0,0,0,354 740,Business,Business,NaN,1.0,1
2,karenslynch,1,10,4,1,0,0,20,18,285 180,Finance,Business,NaN,0.0,6
3,brian-tyler,3,8,25,11,0,0,12,33,29 374,NaN,NaN,NaN,0.0,4
4,steve-collis-632b3,11,11,28,11,0,0,0,0,83 467,Business,Business,NaN,0.0,1


In [5]:
# Create ceo experience columns
df_exp["months_ceo_company"] = df_exp["CEO years"] * 12 + df_exp["CEO months"]
df_exp["months_company"] = df_exp["Overall Comp. years"] * 12 + df_exp["Overall Comp. months"]
df_exp["months_company_others"] = df_exp["Other Comp. years"] * 12 + df_exp["Other Comp. months"]

In [6]:
# Merge experience data with kaggle data
df = df_links.merge(df_exp, left_on="handle", right_on="Handle", how="inner")
df.shape

(507, 39)

In [7]:
# Filter out all junk columns in the data
df.columns

Index(['company', 'rank', 'rank_change', 'revenue', 'profit',
       'num. of employees', 'sector', 'city', 'state', 'newcomer',
       'ceo_founder', 'ceo_woman', 'profitable', 'prev_rank', 'CEO', 'Website',
       'Ticker', 'Market Cap', 'username', 'profile_links', 'handle', 'Handle',
       'CEO years', 'CEO months', 'Overall Comp. years',
       'Overall Comp. months', 'Other CEO years', 'Other CEO months',
       'Other Comp. years', 'Other Comp. months', 'Followers', 'BCs', 'MA',
       'PHD', 'IS_HARVARD', 'Num Comp.', 'months_ceo_company',
       'months_company', 'months_company_others'],
      dtype='object')

In [8]:
columns_to_leave = [
    "company", "rank", "rank_change", "revenue",
    "profit", "num. of employees", "sector", "ceo_woman",
    "CEO", "Ticker", "Market Cap", "handle", "months_ceo_company",
    "months_company", "months_company_others", "Followers", "Num Comp."
]

df = df[columns_to_leave]
df.columns

Index(['company', 'rank', 'rank_change', 'revenue', 'profit',
       'num. of employees', 'sector', 'ceo_woman', 'CEO', 'Ticker',
       'Market Cap', 'handle', 'months_ceo_company', 'months_company',
       'months_company_others', 'Followers', 'Num Comp.'],
      dtype='object')

In [9]:
# Rename columns to lowercase with underscores
df = df.rename(columns={
    "num. of employees": "num_employees",
    "CEO": "ceo",
    "Ticker": "ticker",
    "Market Cap": "market_cap",
    "Followers": "num_followers",
    "Num Comp.": "num_comp"
})

df.columns

Index(['company', 'rank', 'rank_change', 'revenue', 'profit', 'num_employees',
       'sector', 'ceo_woman', 'ceo', 'ticker', 'market_cap', 'handle',
       'months_ceo_company', 'months_company', 'months_company_others',
       'num_followers', 'num_comp'],
      dtype='object')

In [10]:
# Check dtypes 
df.num_followers = df.num_followers.str.replace(r"[^\d]", "", regex=True)
df.num_followers = df.num_followers.astype("int32")

In [11]:
# df.market_cap = df.market_cap.astype("float32")
df.market_cap = df.market_cap.replace("-", np.nan)
df.market_cap = df.market_cap.astype("float32")
df.ceo_woman = df.ceo_woman.map({"yes": 1, "no": 0})

In [12]:
df.dtypes

company                   object
rank                       int64
rank_change              float64
revenue                  float64
profit                   float64
num_employees            float64
sector                    object
ceo_woman                  int64
ceo                       object
ticker                    object
market_cap               float32
handle                    object
months_ceo_company         int64
months_company             int64
months_company_others      int64
num_followers              int32
num_comp                   int64
dtype: object

In [13]:
# Profit_revenue ratio
df["profit_ratio"] = df["profit"] / df["revenue"]

<h3>Collect data from MorningStar website</h3>
<ol>
    <li>CEOs age</li>
    <li>CEOs compensation</li>
</ol>

In [14]:
# Use tickers of the company to find data on CEOs from MorningStar
tickers = df.ticker

In [15]:
# Reverse-engineer REST API of MorningStar to extract json data from server
# Shouldof used asynchronous code but no time for this ;(
class MorningStar:
    
    BASE_URL = "https://api-global.morningstar.com/sal-service/v1/stock/insiders/keyExecutives/{}/" \
    "data?languageId=en&locale=en&clientId=MDC&component=sal-components-executive&version=3.79.0"
    EXCHANGE_QUERY = "https://www.morningstar.com/api/v1/search/securities?q={}&region=USA&limit=5"
    
    def __init__(self):
        self._load_headers()
        self._update_cookies()
    
    # Load real headers from pickle file in src folder
    def _load_headers(self):
        with open("src/headers_ticker.pickle", "rb") as file:
            self.headers_ticker = pickle.load(file)
        with open("src/headers_exchange.pickle", "rb") as file:
            self.headers_exchange = pickle.load(file)
        
    # Use more user-like cookies and headers
    def _update_cookies(self):
        
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get("https://www.morningstar.com/stocks/xnys/wmt/executive")
        
        all_cookies = driver.get_cookies()
        driver.close()
        
        cookies_dict = {}
        for cookie in all_cookies:
            cookies_dict[cookie['name']] = cookie['value']
        
        self.cookies = cookies_dict
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Finished executing... Terminating...")
        
    def _get_exchange(self, ticker: str):
        response = requests.get(
            self.EXCHANGE_QUERY.format(ticker),
            cookies=self.cookies,
            headers=self.headers_exchange
        ).json()
        # Check if there is exchange ticker is traded on
        if len(response["results"]) > 0:
            return response["results"][0]["exchange"].lower()
        else:
            return
        
        
    # Each ticker is masked as id stored in the backend, it appears 
    # once in hmml dom once we query the page with ticker itself
    def get_security_id(self, ticker: str):
        # Each stock is traded on different exchanges either NYSE or NASDAQ or PINX, etc.
        exchange = self._get_exchange(ticker)
        response = requests.get(
            f"https://www.morningstar.com/stocks/{exchange}/{ticker.lower()}", 
            cookies=self.cookies,
            headers=self.headers_ticker
        )
        if response.status_code == 404:
            print(f"Unsuccessful connection with ticker {ticker}")
            return
        # if security id was located then ticker exists on the website -> return it
        security_id = re.findall('<sal-components.*security-id="(.*?)"', response.text)[0]
        return security_id
    
    def request_data(self, ticker: str):
        security_id = self.get_security_id(ticker) # -> get security id to parse data
        if not security_id: # -> if security id is null return null as well
            return 
        data = requests.get(
            self.BASE_URL.format(security_id),
            cookies=self.cookies,
            headers=self.headers_ticker
        ).json()
        # if there are rows -> means there is executives data
        if len(data["rows"]) > 0:
            print(f"Successfully retrieved data for {ticker}")
            return data["rows"][0] 
        # otherwise return null
        print(f"Query went through successfully but no data was retrieved: {ticker}")
        return

In [16]:
data_morning = {
    "ticker": [],
    "data": [],
}

In [17]:
# Parse for each ticker
with MorningStar() as star:
    for ticker in tickers:
        if isinstance(ticker, str): # check if ticker is nan 
            data = star.request_data(ticker)
            data_morning["ticker"].append(ticker)
            data_morning["data"].append(data)
        else:
            data_morning["ticker"].append(np.nan)
            data_morning["data"].append(np.nan)

Successfully retrieved data for WMT
Successfully retrieved data for AMZN
Successfully retrieved data for CVS
Successfully retrieved data for MCK
Successfully retrieved data for ABC
Successfully retrieved data for T
Successfully retrieved data for MSFT
Successfully retrieved data for CAH
Successfully retrieved data for CVX
Successfully retrieved data for WBA
Successfully retrieved data for ELV
Successfully retrieved data for KR
Successfully retrieved data for JPM
Successfully retrieved data for GM
Successfully retrieved data for PSX
Successfully retrieved data for DELL
Successfully retrieved data for FNMA
Successfully retrieved data for UPS
Successfully retrieved data for LOW
Successfully retrieved data for JNJ
Successfully retrieved data for HUM
Successfully retrieved data for PFE
Successfully retrieved data for C
Successfully retrieved data for PEP
Successfully retrieved data for INTC
Successfully retrieved data for PG
Successfully retrieved data for IBM
Successfully retrieved data fo

<h3>Transform collected data</h3>

In [26]:
df_morning = pd.DataFrame(data_morning)

df_morning["age"] = df_morning.data.apply(lambda x: x["age"] if isinstance(x, dict) else np.nan)
df_morning["total_compensation"] = df_morning.data.apply(
    lambda x: x["totalCompensation"][-1] if isinstance(x, dict) else np.nan
)
df_morning["salary"] = df_morning.data.apply(
    lambda x: x["compensation"][0]["datum"][-1] if isinstance(x, dict) else np.nan
)
df_morning["restricted_stock"] = df_morning.data.apply(
    lambda x: x["compensation"][3]["datum"][-1] if isinstance(x, dict) else np.nan
)

df_morning.age = df_morning.age.astype("float32")
df_morning.total_compensation = df_morning.total_compensation.astype("float32")
df_morning.salary = df_morning.salary.astype("float32")
df_morning.restricted_stock = df_morning.restricted_stock.astype("float32")
df_morning.ticker = df_morning.ticker.str.strip()

df_morning.head()

,ticker,data,age,total_compensation,salary,restricted_stock
0,WMT,"{'type': 'person', 'personId': 'PS0000036J', '...",55.0,25670672.0,1276892.0,19195008.0
1,AMZN,"{'type': 'person', 'personId': 'PS000001JW', '...",54.0,212701168.0,175000.0,211933520.0
2,CVS,"{'type': 'person', 'personId': 'PS0000244H', '...",59.0,7045167.0,1429167.0,NaN
3,MCK,"{'type': 'person', 'personId': 'PS0000030D', '...",55.0,18152082.0,1375000.0,12250438.0
4,ABC,"{'type': 'person', 'personId': 'PS00001MO0', '...",60.0,14873815.0,1400000.0,10500201.0


<h3>Merge data from data_300 dataset to collected data</h3>

In [27]:
df_morning = df_morning.drop(["data"], axis=1)
df_morning.shape

(507, 5)

In [28]:
df.reset_index(inplace=True, drop=True)
df_morning.reset_index(inplace=True, drop=True)

df = pd.concat([df, df_morning], axis=1)
df.head()

,company,rank,rank_change,revenue,profit,num_employees,sector,ceo_woman,ceo,ticker,...,ticker,age,total_compensation,salary,restricted_stock,ticker,age,total_compensation,salary,restricted_stock
0,Walmart,1,0.0,572754.0,13673.0,2300000.0,Retailing,0,C. Douglas McMillon,WMT,...,WMT,55.0,25670672.0,1276892.0,19195008.0,WMT,55.0,25670672.0,1276892.0,19195008.0
1,Amazon,2,0.0,469822.0,33364.0,1608000.0,Retailing,0,Andrew R. Jassy,AMZN,...,AMZN,54.0,212701168.0,175000.0,211933520.0,AMZN,54.0,212701168.0,175000.0,211933520.0
2,CVS Health,4,0.0,292111.0,7910.0,258000.0,Health Care,1,Karen Lynch,CVS,...,CVS,59.0,7045167.0,1429167.0,NaN,CVS,59.0,7045167.0,1429167.0,NaN
3,McKesson,9,-2.0,238228.0,-4539.0,67500.0,Health Care,0,Brian S. Tyler,MCK,...,MCK,55.0,18152082.0,1375000.0,12250438.0,MCK,55.0,18152082.0,1375000.0,12250438.0
4,AmerisourceBergen,10,-2.0,213988.8,1539.9,40000.0,Health Care,0,Steven H. Collis,ABC,...,ABC,60.0,14873815.0,1400000.0,10500201.0,ABC,60.0,14873815.0,1400000.0,10500201.0


In [29]:
df.to_csv("output_data/project_data.csv", index=False)